In [223]:
import pandas as pd
import numpy as np
import re
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [276]:
df_train = pd.read_excel('data/Data_Train.xlsx')

In [277]:
df_train.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [278]:
df_train['Destination'].replace({'New Delhi' : 'Delhi'}, inplace = True)

In [279]:
df_train.Total_Stops.fillna('non-stop', inplace = True)

In [280]:
df_train['Total_Stops'].replace({'non-stop' : '0 stop'}, inplace = True)

In [281]:
df_train.Route.fillna('BLR → DEL', inplace = True)

In [282]:
df_train['Additional_Info'].replace({'No info' : 'No Info'}, inplace = True)

In [283]:
from sklearn.preprocessing import LabelEncoder

In [284]:
la = LabelEncoder()
la.fit(df_train.Airline)
df_train.Airline = la.transform(df_train.Airline)

In [285]:
lc = LabelEncoder()
lc.fit(['Banglore', 'Chennai', 'Delhi', 'Kolkata', 'Cochin', 'Hyderabad', 'Mumbai'])
df_train.Source = lc.transform(df_train.Source)
df_train.Destination = lc.transform(df_train.Destination)

In [286]:
lts = LabelEncoder()
lts.fit(['0 stop', '1 stop', '2 stops', '3 stops', '4 stops', '5 stops'])
df_train.Total_Stops = lts.transform(df_train.Total_Stops)

In [287]:
lai = LabelEncoder()
lai.fit(df_train.Additional_Info)
df_train.Additional_Info = lai.transform(df_train.Additional_Info)

In [288]:
df_train.Route = df_train.Route.str.split(' → ')

In [289]:
df_train.Route[0][0]

'BLR'

In [290]:
city_code = set()
for i in range(len(df_train)):
    del df_train.Route[i][0]
    del df_train.Route[i][-1]
    for j in range(len(df_train.Route[i])):
        city_code.add(df_train.Route[i][j])

In [291]:
len(city_code)

42

In [292]:
df_train['Duration'].replace({'5m' : '0h 5m'}, inplace=True)

In [293]:
hour = df_train.Duration.str.split('h')
for i in range (len(hour)):
    if((hour[i][1]) == ''):
        hour[i][1] = ' 0m'

In [294]:
for i in range (len(hour)):
    hour[i][0] = int(hour[i][0])*60
    hour[i][1] = int(hour[i][1].split('m')[0])
    hour[i][0] = hour[i][0] + hour[i][1]

In [295]:
for i in range (len(hour)):
    hour[i] = hour[i][0]
df_train.Duration = hour.astype(int)

In [296]:
dep = df_train.Dep_Time.str.split(':')
for i in range(len(dep)):
    dep[i] = dep[i][0] + dep[i][1]
df_train.Dep_Time = dep.astype(int)

In [297]:
arr = df_train.Arrival_Time.str.split(' ')
for i in range (len(arr)):
    arr[i] = arr[i][0]
arr = arr.str.split(':')
for i in range(len(arr)):
    arr[i] = arr[i][0] + arr[i][1]
df_train.Arrival_Time = arr.astype(int)

In [298]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)


In [299]:
add_datepart(df_train, "Date_of_Journey")

In [300]:
df_train.drop(columns='Route', inplace=True)

In [218]:
#df_train_tmp = df_train

In [219]:
#df_train_con = pd.concat([df_train_tmp,pd.DataFrame(columns=list(city_code))], sort=False)

In [220]:
#df_train_con.fillna(value=False, inplace=True)

In [ ]:
#for i in range (len(df_train_con)):
#    for j in range (len(df_train_con.Route[i])):
#        x = df_train_con.Route[i][j]
#        df_train_con[x][i] = True

In [209]:
#df_train_con['IXR'].head()

0    False
1     True
2    False
3    False
4    False
Name: IXR, dtype: object

In [301]:
df_train.head(10)

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date_of_JourneyYear,...,Date_of_JourneyDay,Date_of_JourneyDayofweek,Date_of_JourneyDayofyear,Date_of_JourneyIs_month_end,Date_of_JourneyIs_month_start,Date_of_JourneyIs_quarter_end,Date_of_JourneyIs_quarter_start,Date_of_JourneyIs_year_end,Date_of_JourneyIs_year_start,Date_of_JourneyElapsed
0,3,0,3,2220,110,170,0,6,3897,2019,...,24,6,83,False,False,False,False,False,False,1553385600
1,1,5,0,550,1315,445,2,6,7662,2019,...,1,2,121,False,True,False,False,False,False,1556668800
2,4,3,2,925,425,1140,2,6,13882,2019,...,9,6,160,False,False,False,False,False,False,1560038400
3,3,5,0,1805,2330,325,1,6,6218,2019,...,12,6,132,False,False,False,False,False,False,1557619200
4,3,0,3,1650,2135,285,1,6,13302,2019,...,1,4,60,False,True,False,False,False,False,1551398400
5,8,5,0,900,1125,145,0,6,3873,2019,...,24,0,175,False,False,False,False,False,False,1561334400
6,4,0,3,1855,1025,930,1,5,11087,2019,...,12,1,71,False,False,False,False,False,False,1552348800
7,4,0,3,800,505,1265,1,6,22270,2019,...,1,4,60,False,True,False,False,False,False,1551398400
8,4,0,3,855,1025,1530,1,5,11087,2019,...,12,1,71,False,False,False,False,False,False,1552348800
9,6,3,2,1125,1915,470,1,6,8625,2019,...,27,0,147,False,False,False,False,False,False,1558915200


In [302]:
df_train.dtypes

Airline                            int64
Source                             int64
Destination                        int64
Dep_Time                           int64
Arrival_Time                       int64
Duration                           int64
Total_Stops                        int64
Additional_Info                    int64
Price                              int64
Date_of_JourneyYear                int64
Date_of_JourneyMonth               int64
Date_of_JourneyWeek                int64
Date_of_JourneyDay                 int64
Date_of_JourneyDayofweek           int64
Date_of_JourneyDayofyear           int64
Date_of_JourneyIs_month_end         bool
Date_of_JourneyIs_month_start       bool
Date_of_JourneyIs_quarter_end       bool
Date_of_JourneyIs_quarter_start     bool
Date_of_JourneyIs_year_end          bool
Date_of_JourneyIs_year_start        bool
Date_of_JourneyElapsed             int64
dtype: object

In [303]:
df_train['modPrice'] = np.log(df_train.Price + 1)

In [304]:
from sklearn.model_selection import train_test_split

In [315]:
def rmsle(y,y0):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))


In [306]:
train, test = train_test_split(df_train, test_size=0.3)

In [317]:
model = CatBoostRegressor(iterations=500, learning_rate=0.2, depth = 8, loss_function='RMSE')
model.fit(train.drop(columns = ['Price', 'modPrice'], axis=1), train.modPrice, )

0:	learn: 7.2196247	total: 28.5ms	remaining: 14.2s
1:	learn: 5.7960661	total: 71.2ms	remaining: 17.7s
2:	learn: 4.6477075	total: 83ms	remaining: 13.7s
3:	learn: 3.7370917	total: 103ms	remaining: 12.8s
4:	learn: 3.0013513	total: 110ms	remaining: 10.9s
5:	learn: 2.4119332	total: 127ms	remaining: 10.5s
6:	learn: 1.9421413	total: 135ms	remaining: 9.49s
7:	learn: 1.5724168	total: 154ms	remaining: 9.49s
8:	learn: 1.2786960	total: 173ms	remaining: 9.41s
9:	learn: 1.0405667	total: 191ms	remaining: 9.38s
10:	learn: 0.8520672	total: 195ms	remaining: 8.65s
11:	learn: 0.7048939	total: 213ms	remaining: 8.67s
12:	learn: 0.5915281	total: 240ms	remaining: 8.97s
13:	learn: 0.5039718	total: 270ms	remaining: 9.38s
14:	learn: 0.4398556	total: 274ms	remaining: 8.86s
15:	learn: 0.3859878	total: 282ms	remaining: 8.52s
16:	learn: 0.3357261	total: 304ms	remaining: 8.65s
17:	learn: 0.3093682	total: 312ms	remaining: 8.34s
18:	learn: 0.2841893	total: 329ms	remaining: 8.32s
19:	learn: 0.2717831	total: 333ms	remain

194:	learn: 0.1883199	total: 1.66s	remaining: 2.59s
195:	learn: 0.1883198	total: 1.66s	remaining: 2.58s
196:	learn: 0.1883198	total: 1.67s	remaining: 2.57s
197:	learn: 0.1883197	total: 1.68s	remaining: 2.56s
198:	learn: 0.1883197	total: 1.69s	remaining: 2.55s
199:	learn: 0.1883197	total: 1.7s	remaining: 2.54s
200:	learn: 0.1882975	total: 1.72s	remaining: 2.56s
201:	learn: 0.1882975	total: 1.73s	remaining: 2.55s
202:	learn: 0.1882974	total: 1.73s	remaining: 2.54s
203:	learn: 0.1882974	total: 1.74s	remaining: 2.53s
204:	learn: 0.1882974	total: 1.75s	remaining: 2.52s
205:	learn: 0.1882974	total: 1.76s	remaining: 2.52s
206:	learn: 0.1882974	total: 1.77s	remaining: 2.51s
207:	learn: 0.1882974	total: 1.79s	remaining: 2.51s
208:	learn: 0.1882974	total: 1.81s	remaining: 2.52s
209:	learn: 0.1882974	total: 1.82s	remaining: 2.51s
210:	learn: 0.1882974	total: 1.83s	remaining: 2.5s
211:	learn: 0.1882974	total: 1.84s	remaining: 2.5s
212:	learn: 0.1882974	total: 1.84s	remaining: 2.49s
213:	learn: 0.1

365:	learn: 0.1789596	total: 3.26s	remaining: 1.19s
366:	learn: 0.1788931	total: 3.29s	remaining: 1.19s
367:	learn: 0.1788931	total: 3.29s	remaining: 1.18s
368:	learn: 0.1788931	total: 3.3s	remaining: 1.17s
369:	learn: 0.1788931	total: 3.31s	remaining: 1.16s
370:	learn: 0.1788931	total: 3.31s	remaining: 1.15s
371:	learn: 0.1788930	total: 3.31s	remaining: 1.14s
372:	learn: 0.1788930	total: 3.31s	remaining: 1.13s
373:	learn: 0.1788930	total: 3.32s	remaining: 1.12s
374:	learn: 0.1788930	total: 3.32s	remaining: 1.11s
375:	learn: 0.1788930	total: 3.32s	remaining: 1.09s
376:	learn: 0.1788930	total: 3.33s	remaining: 1.08s
377:	learn: 0.1788788	total: 3.34s	remaining: 1.08s
378:	learn: 0.1788788	total: 3.35s	remaining: 1.07s
379:	learn: 0.1788787	total: 3.36s	remaining: 1.06s
380:	learn: 0.1788787	total: 3.37s	remaining: 1.05s
381:	learn: 0.1788787	total: 3.38s	remaining: 1.04s
382:	learn: 0.1788787	total: 3.38s	remaining: 1.03s
383:	learn: 0.1788787	total: 3.39s	remaining: 1.02s
384:	learn: 0

In [318]:
pred = model.predict(test.drop(columns = ['Price', 'modPrice'], axis=1))

In [319]:
mp = np.exp(pred) + 1

In [320]:
mp

array([12836.69847178,  3765.47381175,  2436.52275729, ...,
        4381.95890848, 12011.02900378, 10873.1184175 ])

In [321]:
rmsle(test.Price, mp)

0.1817214437695527